1. 港股的汇率问题;
2. 使用不复权的数据和除权除息的数据,定点复权

### 港股的汇率问题

In [2]:
# 首先是拿到HKD/USD的数据
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib as ta
import datetime, time
import pickle
from tqdm import tqdm_notebook

import pdblp as pb
import blpapi

con = pb.BCon(port=8194, timeout=5000)
# con.start()

code = 'USDHKD BGNL Curncy'
start = '20100101'
end = '20190117'
# df = con.bdh(code, ['PX_LAST'], start, end)

In [4]:
with open('data/df_cur', 'rb') as f:
    df = pickle.load(f)
df.head()

ticker,USDHKD BGNL Curncy
field,PX_LAST
date,
2010-01-01,7.7552
2010-01-04,7.7554
2010-01-05,7.7569
2010-01-06,7.7547
2010-01-07,7.7546


In [5]:
df.tail()

ticker,USDHKD BGNL Curncy
field,PX_LAST
date,
2019-01-11,7.8394
2019-01-14,7.8425
2019-01-15,7.8433
2019-01-16,7.8431
2019-01-17,7.8432


```
ticker	USDHKD BGNL Curncy
field	PX_LAST
date	
2019-01-21	7.8448
2019-01-22	7.8455
2019-01-23	7.8462
2019-01-24	7.8458
2019-01-25	7.8466
```

In [7]:
df[code].head()

field,PX_LAST
date,
2010-01-01,7.7552
2010-01-04,7.7554
2010-01-05,7.7569
2010-01-06,7.7547
2010-01-07,7.7546


In [8]:
# 然后对港股的行情数据进行处理
from utils import *

In [9]:
data = ol_data.copy()

In [10]:
hk_code = df_code.query('region == "HK Equity"').symbol.unique().tolist()

In [14]:
# hk_code

In [25]:
df.index[0].strftime('%Y-%m-%d')

'2010-01-01'

In [13]:
data.loc[(slice(None), hk_code),:].head()

high    open     low     volume  Adj Close
date       code                                              
2010-01-04 1149   0.615   0.605   0.530  5244000.0      0.558
           1385   1.583   1.483   1.465  1420000.0      1.574
           148   18.469  18.171  17.725   966247.0     17.934
           1686   0.336   0.319   0.319  8601612.0      0.332
           1888   3.295   3.150   3.110  6502000.0      3.254

In [30]:
df.iloc[0][0]

7.7552

In [35]:
# 指定日期
for i in df.index[:5]:
    time_str = i.strftime('%Y-%m-%d')
#     data.loc[(time_str, hk_code),:] = data.loc[(time_str, hk_code),:] / df[i][0]
    print((data.loc[(time_str, hk_code),:] / df.loc[i][0]).head())
#     print(data.loc[(time_str, hk_code),:].head() )

# save data

Empty DataFrame
Columns: [high, open, low, volume, Adj Close]
Index: []
                     high      open       low        volume  Adj Close
date       code                                                       
2010-01-04 1149  0.079300  0.078010  0.068339  6.761740e+05   0.071950
           1385  0.204116  0.191222  0.188901  1.830982e+05   0.202955
           148   2.381437  2.343013  2.285504  1.245902e+05   2.312453
           1686  0.043325  0.041133  0.041133  1.109113e+06   0.042809
           1888  0.424865  0.406169  0.401011  8.383836e+05   0.419579
                     high      open       low         volume  Adj Close
date       code                                                        
2010-01-05 1149  0.074385  0.073096  0.070647  298572.883497   0.074385
           1385  0.209878  0.205237  0.199306  242364.862252   0.207557
           148   2.384844  2.315745  2.315745  132032.899741   2.373242
           1686  0.043316  0.043316  0.042414  496362.722222   0.042801

### 使用不复权的数据和除权除息的数据,定点复权

#### 拿到不复权的数据

```python
# 修改utils_update.py
temp_df = con.bdh(temp_list, fields, start, end)
# 改为
elements = [('CshAdjAbnormal', False), ('CapChg', False), ('CshAdjNormal', False)]
temp_df = con.bdh(temp_list, fields, start, end, elms=elements)
```

#### 获取除权除息的数据

```excel
=BDS("AAPL US Equity","DVD_HIST_ALL","headers=y","cols=7;rows=67")
```

In [15]:
code = "AAPL US Equity"
fields = "DVD_HIST_ALL"
df_dvd = con.bulkref(code, fields)
df_dvd

In [ ]:
code = "AAPL US Equity"
fields = "DVD_HIST_ALL"
overrides = {'headers':True}
df_dvd_2 = con.bulkref(code, fields, ovrds=overrides)
df_dvd_2

成功拿到这些数据之后，就是要考虑定点复权的问题

还有数据更新的问题，以及他想加入新的code和etf的问题

In [1]:
from utils_v2 import *

In [37]:
res1 = calc_pair_trading('SMH', period=245, rt_period=255)
res2 = calc_pair_trading('SMH', period=1, rt_period=255)

In [38]:
res1['long_return'].sum().sum()

-0.11748640146501806

In [39]:
res2['long_return'].sum().sum()

-0.15015685089676106

In [40]:
res1['long_short_return'].sum().sum()

-0.02789620492814109

In [41]:
res2['long_short_return'].sum().sum()

-0.09323710379162711

In [ ]:
res3 = calc_pair_trading('PNQI', period=245, rt_period=255)

BKNG or IUGXX don't have data, please check
GOOG or IUGXX don't have data, please check
FB or IUGXX don't have data, please check
AMZN or IUGXX don't have data, please check
NFLX or IUGXX don't have data, please check
AABA or IUGXX don't have data, please check
EQIX or IUGXX don't have data, please check
EBAY or IUGXX don't have data, please check
BIDU or IUGXX don't have data, please check


In [6]:
ol_data.loc[(slice('2015','2019'), slice(None)), :]

high        open         low      volume   Adj Close
date       code                                                              
2015-01-02 036570  173379.000  172901.000  169558.000    171149.0  172424.000
           053800   34946.000   34753.000   34174.000     42121.0   34657.000
           069080   30200.000   28500.000   28100.000   1988995.0   28700.000
           078340  128112.000  117080.000  116738.000    841240.0  123922.000
           095660   23100.000   22650.000   22200.000    201636.0   22900.000
           1089         0.850       0.850       0.820  11905000.0       0.850
           112040   37970.000   36338.000   36338.000    215380.0   37922.000
           1149         0.510       0.490       0.490   3408000.0       0.495
           1236         2.680       2.610       2.610   2000000.0       2.680
           1347         9.212       9.212       8.843    141000.0       9.046
           1385         6.790       6.790       5.900    300000.0       6.160
           148         10.928      10.797      10.699    691900.0      10.912
           1686         2.127       2.092       2.092    741000.0       2.101
           1888         2.103       2.066       2.066   4864000.0       2.095
           1980         3.898       3.898       3.661   1445000.0       3.718
           2018        38.607      38.515      37.678    673485.0      38.282
           2280         7.498       7.122       7.122   2462000.0       7.449
           2342         1.813       1.794       1.735   7832650.0       1.748
           2369         1.546       1.536       1.497   9087173.0       1.516
           2382        13.134      13.076      12.611    852400.0      12.786
           2662         0.555       0.537       0.519   1434000.0       0.528
           268          2.358       2.288       2.278   3120800.0       2.338
           285          7.563       7.179       7.179   3675000.0       7.496
           31           0.912       0.856       0.856  16198600.0       0.912
           327          7.683       7.541       7.417    760000.0       7.427
           3336         3.317       3.233       3.216   1836000.0       3.292
           3393         6.218       6.005       5.980    968000.0       6.120
           354          2.228       2.138       2.128   4092000.0       2.198
           3800         1.709       1.663       1.653  56156930.0       1.690
           3888        15.382      15.127      15.107   3184000.0      15.186
...                       ...         ...         ...         ...         ...
2019-01-17 61           0.053       0.053       0.052   2400000.0       0.053
           6111       124.500     123.000     119.000   1126000.0     122.000
           6180        78.400      75.500      75.400   7723000.0      78.400
           633          0.360       0.345       0.345    874000.0       0.350
           6460      1578.000    1569.000    1556.000    656800.0    1573.000
           6869        23.050      22.550      22.500   1777000.0      22.600
           696         20.500      20.350      20.100    834000.0      20.350
           698          0.770       0.760       0.750   3340000.0       0.760
           700        338.000     335.000     330.600  15716301.0     336.200
           732          0.940       0.910       0.900   3317580.0       0.930
           763         15.780      15.600      15.340   4721453.0      15.400
           777         14.120      14.000      13.720    321500.0      13.840
           7832      4750.000    4630.000    4620.000    892200.0    4715.000
           7844       884.000     872.000     871.000    104100.0     880.000
           7974     33250.000   32370.000   32150.000   2564600.0   33090.000
           799         11.720      11.480      11.320   6132732.0      11.640
           8083         0.560       0.540       0.540   7524000.0       0.560
           818          1.130       1.110       1.100   1518000.0       1.130
           8227         3.050       3.050      